# Ch08. 딥 러닝(Deep Learning) 개요

# v07. 다층 퍼셉트론(Multi Layer Perceptron, MLP)으로 텍스트 분류하기

## 7.1 다층 퍼셉트론(Multi Layer Perceptron, MLP)

- 단층 퍼셉트론의 형태에서 은닉층이 1개 이상 추가된 신경망을 **다층 퍼셉트론(MLP)**이라고 한다.
- 다층 퍼셉트론은 **피드 포워드 신경망(Feed Forward Neural Network, FFNN)**의 가장 기본적인 형태이다.  
(피드 포워드 신경망 : 입력층에서 출력층으로 오직 한 방향으로만 연산 방향이 정해져 있는 신경망)

- 뒤에서는 다음 2가지의 새로운 개념들을 사용하여 자연어 처리 실습을 한다.
  - 순환 신경망(RNN)
  - 분산 표현(distributed representation)
- 이번 챕터에서는 다음 두 가지 개념 없이 지금까지 배운 개념만으로도 자연어 처리를 할 수 있다는 것을 보여준다.

<br>

## 7.2 케라스의 `texts_to_matrix()` 이해하기

- MLP로 텍스트 분류를 수행하기 전에 이번에 사용할 도구인 케라스의 `Tokenizer`의 `texts_to_matrix()`를 이해해보자.

<br>

### 7.2.1 라이브러리 임포트

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

In [0]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

<br>

### 7.2.2 텍스트 데이터

- 여기에서 사용할 텍스트 데이터는 다음 네 문장이다.

In [0]:
texts = ['먹고 싶은 사과',
         '먹고 싶은 바나나',
         '길고 노란 바나나 바나나',
         '저는 과일이 좋아요']

<br>

### 7.2.3 정수 인코딩

- 위의 텍스트 데이터에 대해서 정수 인코딩을 수행한다.
- `Tokenizer().fit_on_texts()`를 이용해 텍스트 데이터를 단어들로 분리한 뒤, 각 단어에 정수 인덱스를 부여한다.

In [4]:
t = Tokenizer()
t.fit_on_texts(texts)
print(t.word_index)

{'바나나': 1, '먹고': 2, '싶은': 3, '사과': 4, '길고': 5, '노란': 6, '저는': 7, '과일이': 8, '좋아요': 9}


- 각 단어에 숫자 1부터 시작하는 정수 인덱스가 부여됐다.

<br>

### 7.2.4 `texts_to_matrix()` 사용

- 이 도구는 입력된 텍스트 데이터로부터 행렬(matrix)을 만드는 도구이다.
- `texts_to_matrix()`는 다음 4가지의 모드를 지원한다.
  - `mode='binary'`
  - `mode='count'`
  - `mode='freq'`
  - `mode='tfidf'`

<br>

#### 7.2.4.1 `texts_to_matrix(mode='count')`

- `'count'` 모드를 사용하면 앞서 배운 **문서 단어 행렬(Document-Term Matrix, DTM)**을 생성한다.
- DTM에서의 인덱스는 앞서 확인한 `word_index`의 결과이다.

In [5]:
print(t.texts_to_matrix(texts, mode='count'))

[[0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


**주의할 점**

- 각 단어에 부여되는 인덱스는 1부터 시작하는 반면, 완성되는 행렬의 인덱스는 0부터 시작한다.
- 실제로 단어의 개수는 9개였지만, 
  - 완성된 행렬의 열의 개수는 10개
  - 첫 번째 열은 모든 행에서 값이 0  
  (인덱스 0에는 그 어떤 단어도 할당되지 않았기 때문)

**네 번째 행**

- 네 번째 행은 테스트 데이터에서 네 번째 문장을 의미한다.
- 네번째 행은 8번째 열, 9번째 열, 10번째 열에서 1의 값을 가진다.
- 이는 7번 단어, 8번 단어, 9번 단어가 네번째 문장에서 1개씩 존재함을 의미한다.
- 위에서 정수 인코딩 된 결과를 보면 7번 단어는 '저는', 8번 단어는 '과일이', 9번 단어는 '좋아요'이다.

**세 번째 행**

- 세 번째 행의 첫 번째 열의 값은 2이다.
- 이는 세 번째 문장에서 1번 인덱스를 가진 바나나가 두 번 등장했기 때문이다.

**단어 순서 정보**

- DTM은 bag of words를 기반으로 하므로 **단어의 순서 정보는 보존되지 않는다.**
- 사실 더 구체적으로 4개의 모든 모드에서 단어 순서 정보는 보존되지 않는다.

<br>

#### 7.2.4.2 `texts_to_matrix(mode='binary')`

In [6]:
print(t.texts_to_matrix(texts, mode='binary'))

[[0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


- DTM과 결과가 매우 유사해보인다.
- 다만 세 번째 행의 두 번째 열의 값이 DTM에서는 2였는데 여기서는 1로 바뀌었다.
- 그 이유는 `'binary'` 모드는 해당 단어가 존재하는 지만 관심을 가지고 해당 단어가 몇 개 였는 지는 무시하기 때문이다.
- 해당 단어가 존재하면 1, 단어가 존재하지 않으면 0의 값을 가진다.
- 즉, **단어의 존재 유무로만 행렬을 표현**한다.

<br>

#### 7.2.4.3 `texts_to_matrix(mode='tfidf')`

In [7]:
print(t.texts_to_matrix(texts, mode='tfidf').round(2))

[[0.   0.   0.85 0.85 1.1  0.   0.   0.   0.   0.  ]
 [0.   0.85 0.85 0.85 0.   0.   0.   0.   0.   0.  ]
 [0.   1.43 0.   0.   0.   1.1  1.1  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.1  1.1  1.1 ]]


- `'tfidf'` 모드는 말 그대로 TF-IDF 행렬을 만든다.
- 다만, TF-IDF 챕터에서 배운 기본식이나 사이킷런의 `TfidfVectorizer`에서 사용하는 식과는 차이가 있다.
  - 앞서 배운 기본식에서의 TF : 각 문서에서의 각 단어의 빈도
  - `'tfidf'` 모드에서의 TF : 각 문서에서의 각 단어의 빈도에 자연 로그를 씌우고 1을 더한 값으로 정의
  - `'tfidf'` 모드에서의 IDF : 기본식에서 로그는 자연 로그를 사용, 로그 안의 분수에 1을 추가로 더함

<br>

#### 7.2.4.4 `texts_to_matrix(mode='freq')`

In [8]:
print(t.texts_to_matrix(texts, mode='freq').round(2))

[[0.   0.   0.33 0.33 0.33 0.   0.   0.   0.   0.  ]
 [0.   0.33 0.33 0.33 0.   0.   0.   0.   0.   0.  ]
 [0.   0.5  0.   0.   0.   0.25 0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.33 0.33 0.33]]


- `'freq'` 모드는 각 문서에서의 각 단어의 등장 횟수를 분자로, 각 문서의 크기(각 문서에서 등장한 모든 단어의 개수의 총합)를 분모로 하는 표현 방법이다.

$
\qquad
\frac{\text{해당 문서에서 각 단어의 등장 횟수}}{\text{해당 문서에 등장한 모든 단어의 개수의 총합}}
$

**세 번째 문장**

> 길고 노란 바나나 바나나

- 문서의 크기 : 4
- '바나나' : 2회 등장 $\rightarrow$ 2 / 4 = 0.5
- '길고' : 1회 등장 $\rightarrow$ 1 / 4 = 0.25
- '노란' : 1회 등장 $\rightarrow$ 1 / 4 = 0.25

<br>

## 7.3 20개 뉴스 그룹(Twenty Newsgroups) 데이터에 대한 이해

### 7.3.1 필요 라이브러리 임포트

In [0]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

<br>

### 7.3.2 데이터셋

- 사이킷런에서는 20개의 다른 주제를 가진 18,846개의 뉴스 그룹 이메일 데이터를 제공한다. (LSA 챕터와 동일한 데이터)

In [10]:
newsdata = fetch_20newsgroups(subset='train') # 훈련 데이터만 리턴

- `subset='all'` : 전체 데이터인 18,846개의 샘플 다운로드
- `subset='train'` : 훈련 데이터 다운로드
- `subset='test'` : 테스트 데이터 다운로드

<br>

### 7.3.3 데이터 속성 확인

- `newsdata.keys()`를 출력하면 해당 데이터의 속성을 확인할 수 있다.

In [11]:
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


- `data` : 이메일 본문
- `target` : 메일이 어떤 주제인 지 기재된 숫자 레이블

<br>

### 7.3.4 훈련용 샘플의 개수 확인

In [12]:
print("훈련용 샘플의 개수 : {}".format(len(newsdata.data)))

훈련용 샘플의 개수 : 11314


<br>

### 7.3.5 20개의 주제 확인

- `target_names`에는 20개의 주제의 이름을 담고 있다.

In [13]:
print('총 주제의 개수 : {}'.format(len(newsdata.target_names)))

총 주제의 개수 : 20


In [14]:
print(newsdata.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
